In [2]:
# Download Libraries
!pip3 install google-cloud-storage
!pip3 install pyarrow # Apache Arrow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Import Libraries
import pandas as pd
import numpy as np
import json
import requests
import zipfile
import io
from io import StringIO
import pyarrow as pa
import pyarrow.parquet as pq
from google.cloud import storage
import os

/Users/karmayangchentenzin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [31]:
#Extracting from datasource to view the head
URL = 'https://data.cityofnewyork.us/api/views/vx8i-nprf/rows.csv?accessType=DOWNLOAD'

# Define the data type for column 17 because it contains mixed data types including NaN
dtype_mapping = {'Veteran Credit': 'object'}

# Specify NaN values to be considered as missing values
na_values = ['NaN', '', 'NA', 'nan']

df_raw = pd.read_csv(URL, dtype=dtype_mapping, na_values=na_values, low_memory=False)

print(df_raw.info())
print('\n')
print(df_raw.shape)
print('\n')
df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491819 entries, 0 to 491818
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Exam No             491819 non-null  int64  
 1   List No             491819 non-null  float64
 2   First Name          491812 non-null  object 
 3   MI                  272572 non-null  object 
 4   Last Name           491811 non-null  object 
 5   Adj. FA             491819 non-null  float64
 6   List Title Code     491819 non-null  int64  
 7   List Title Desc     491819 non-null  object 
 8   Group No            491819 non-null  int64  
 9   List Agency Code    491819 non-null  int64  
 10  List Agency Desc    491819 non-null  object 
 11  List Div Code       0 non-null       float64
 12  Published Date      223254 non-null  object 
 13  Established Date    477619 non-null  object 
 14  Anniversary Date    477619 non-null  object 
 15  Extension Date      227556 non-nul

,Exam No,List No,First Name,MI,Last Name,Adj. FA,List Title Code,List Title Desc,Group No,List Agency Code,List Agency Desc,List Div Code,Published Date,Established Date,Anniversary Date,Extension Date,Veteran Credit,Parent Lgy Credit,Sibling Lgy Credit,Residency Credit
0,9618,1214.0,GERMAN,A,SOSA,86.67,91203,BUS OPERATOR,0,0,OPEN COMPETITIVE,NaN,NaN,09/08/2021,09/08/2025,NaN,NaN,NaN,NaN,NaN
1,7001,24935.0,DARREN,L,PAYNE,91.00,70310,FIREFIGHTER,0,0,OPEN COMPETITIVE,NaN,06/13/2018,02/27/2019,02/27/2023,02/27/2025,NaN,NaN,NaN,Residency Credit
2,7001,11653.0,STEPHEN,NaN,MANFRE,96.00,70310,FIREFIGHTER,0,0,OPEN COMPETITIVE,NaN,06/13/2018,02/27/2019,02/27/2023,02/27/2025,NaN,NaN,NaN,Residency Credit
3,2060,15669.0,ELIJAH,T,RICHARDSON,95.71,70112,SANITATION WORKER,0,0,OPEN COMPETITIVE,NaN,07/12/2023,02/28/2024,02/28/2028,NaN,NaN,NaN,NaN,Residency Credit
4,6601,16886.0,JAY,M,AMES,88.75,91207,CONDUCTOR,0,0,OPEN COMPETITIVE,NaN,NaN,02/14/2018,02/14/2022,02/14/2025,NaN,NaN,NaN,NaN


In [4]:
# Specify the path to your service account key file
service_account_key_file = '/Users/karmayangchentenzin/Downloads/service_account_key.json'

# Optionally, load other configuration settings from the JSON file
with open(service_account_key_file, 'r') as f:
    config = json.load(f)

# Extracting data from datasource to view the head
URL = 'https://data.cityofnewyork.us/api/views/vx8i-nprf/rows.csv?accessType=DOWNLOAD'

# Define the data type for column 17 because it contains mixed data types including NaN
dtype_mapping = {'Veteran Credit': 'object'}

# Specify NaN values to be considered as missing values
na_values = ['NaN', '', 'NA', 'nan']

df_raw = pd.read_csv(URL, dtype=dtype_mapping, na_values=na_values, low_memory=False)

print(df_raw.info())
print('\n')
print(df_raw.shape)
print('\n')
df_raw.head()

# Fetch data from the web (CSV file)
url = 'https://data.cityofnewyork.us/api/views/vx8i-nprf/rows.csv?accessType=DOWNLOAD'

# Read the contents of the CSV file
response = requests.get(url)

# Check if the request was successful
if response.status_code != 200:
    print("Failed to fetch CSV file.")
    exit()

# Read the CSV data into a DataFrame
try: 
    df = pd.read_csv(io.StringIO(response.text), dtype=dtype_mapping, na_values=na_values, low_memory=False)

    # Convert DataFrame to PyArrow Table
    table = pa.Table.from_pandas(df)

    # Write PyArrow Table to Parquet format
    parquet_file_name = 'ny_civil_service_exam.parquet'  # name for the Parquet file
    pq.write_table(table, parquet_file_name)

    # Upload Parquet file to Google Cloud Storage
    bucket_name = 'cis4400_hw1_kyt'
    blob_name = parquet_file_name

    client = storage.Client.from_service_account_json(service_account_key_file)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(parquet_file_name)

    print(f"Parquet file '{parquet_file_name}' uploaded to {bucket_name}/{blob_name} in Google Cloud Storage.")
except Exception as e:
    print(f"Error occurred: {e}")
    


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491819 entries, 0 to 491818
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Exam No             491819 non-null  int64  
 1   List No             491819 non-null  float64
 2   First Name          491812 non-null  object 
 3   MI                  272572 non-null  object 
 4   Last Name           491811 non-null  object 
 5   Adj. FA             491819 non-null  float64
 6   List Title Code     491819 non-null  int64  
 7   List Title Desc     491819 non-null  object 
 8   Group No            491819 non-null  int64  
 9   List Agency Code    491819 non-null  int64  
 10  List Agency Desc    491819 non-null  object 
 11  List Div Code       0 non-null       float64
 12  Published Date      223254 non-null  object 
 13  Established Date    477619 non-null  object 
 14  Anniversary Date    477619 non-null  object 
 15  Extension Date      227556 non-nul